In [63]:
import pandas as pd
import json
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

stopwords_list = stopwords.words("russian")
stopwords_list.extend(stopwords.words("english"))
import pymorphy2

analyzer = pymorphy2.MorphAnalyzer()
from collections import Counter

In [64]:
with open('bios_corpus.json', 'r') as f:
    corpus = json.load(f)

In [65]:
len(corpus)

1604

In [66]:
emoji_pattern = re.compile('['
                           u'\U0001F600-\U0001F64F'
                           u'\U0001F300-\U0001F5FF'
                           u'\U0001F680-\U0001F6FF'
                           u'\U0001F1E0-\U0001F1FF'
                           u'\U0001F1F2-\U0001F1F4'
                           u'\U0001F1E6-\U0001F1FF'
                           u'\U0001F600-\U0001F64F'
                           u'\U00002702-\U000027B0'
                           u'\U000024C2-\U0001F251'
                           u'\U0001f926-\U0001f937'
                           u'\U0001F1F2'
                           u'\U0001F1F4'
                           u'\U0001F620'
                           u'\u200d'
                           u'\u2640-\u2642'
                           u'\U0001F919'  # added emojis
                           u'\U000026E9'
                           u'\U0001F913'
                           u'\U0001F9D0'
                           u'\U00002705'
                           u'\U00002753'
                           u'\U0001F926'
                           u'\U0001F937'
                           u'\U0001F918'
                           u'\U0001F9ED'
                           ']+', flags=re.UNICODE)

url_pattern = re.compile(r'https?://(?:www\.)?(?:[-a-zA-Z0-9@:%._\+~#=/?&]+)')
brackets_id_pattern = re.compile(r'\[\w+\|\w+\-?\w+?\s?\w+\-?\w+?\]')
tag_patten = re.compile(r'@\w+')

In [67]:
def regex(corpus_json):
    edited_corpus = []
    for i, doc in enumerate(corpus_json):
        edited_doc = re.sub('\n', ' ', corpus_json[i])
        edited_doc = emoji_pattern.sub('', edited_doc)
        edited_doc = url_pattern.sub('', edited_doc)
        edited_doc = brackets_id_pattern.sub('', edited_doc)
        edited_doc = tag_patten.sub('', edited_doc)
        edited_corpus.append(edited_doc)


def preprocess(doc: str):
    re_tokenizer = RegexpTokenizer(r'[А-яA-z-ё]+')
    tokens = re_tokenizer.tokenize(doc)
    tokens = [token.lower() for token in tokens if len(token) > 1 and token.isalpha()]
    tokenized_doc = [word for word in tokens if word not in stopwords_list]
    norm_doc = [analyzer.parse(word)[0].normal_form for word in tokenized_doc]
    return norm_doc


def prepare_ner(tokenized_corpus):
    nouns_corpus = []
    for doc in tokenized_corpus:
        nouns_doc = [word for word in doc if analyzer.parse(word)[0].tag.POS == 'NOUN']
        if nouns_doc:
            nouns_corpus.append(nouns_doc)
    return nouns_corpus

In [68]:
tokenized = []
for bio in corpus:
    if bio != '':
        tokenized_bio = preprocess(bio)
        tokenized.append(tokenized_bio)

tokenized[:5]

[['глава',
  'местный',
  'администрация',
  'муниципальный',
  'округ',
  'васильевский',
  'чгэк',
  'спбп',
  'пётр',
  'великий',
  'старший',
  'преподаватель',
  'спбгипср'],
 ['рыбак', 'охотник', 'стаж', 'любой', 'снаряжение', 'лс'],
 ['активный',
  'пенсионер',
  'официальный',
  'партнёр',
  'корпорация',
  'сибирский',
  'здоровье',
  'директор',
  'центр',
  'обслуживание',
  'клиент',
  'коломна'],
 ['настоящий',
  'швеция',
  'её',
  'человек',
  'living',
  'sweden',
  'det',
  'har',
  'nt',
  'riktigt',
  'inspirerad',
  'av',
  'verkliga',
  'ndelser',
  'sverige',
  'malm',
  'sweden'],
 ['руководитель', 'бид', 'главный', 'редактор', 'дизайнер']]

In [69]:
with open('tokenized_corpus.json', 'w') as f:
    json.dump(tokenized, f, ensure_ascii=False)

In [70]:
nouns_corpus = prepare_ner(tokenized)

nouns_corpus[:5]

[['глава',
  'администрация',
  'округ',
  'чгэк',
  'спбп',
  'пётр',
  'старший',
  'преподаватель',
  'спбгипср'],
 ['рыбак', 'охотник', 'стаж', 'снаряжение'],
 ['пенсионер',
  'партнёр',
  'корпорация',
  'здоровье',
  'директор',
  'центр',
  'обслуживание',
  'клиент',
  'коломна'],
 ['швеция', 'человек'],
 ['руководитель', 'редактор', 'дизайнер']]

### most frequent nouns

In [71]:
N_KEYWORDS = 50
flat_corpus = [word for doc in nouns_corpus for word in doc]
counter = Counter(flat_corpus)
counter_sorted = dict(sorted(counter.items(),
                             key=lambda item: item[1], reverse=True))
print(len(counter_sorted))
list(counter_sorted.items())[:N_KEYWORDS]

1414


[('здоровье', 79),
 ('жизнь', 79),
 ('байкал', 51),
 ('красота', 48),
 ('бизнес', 43),
 ('путешествие', 32),
 ('год', 32),
 ('мир', 31),
 ('мама', 29),
 ('человек', 23),
 ('семья', 22),
 ('зож', 21),
 ('ссылка', 20),
 ('иркутск', 20),
 ('директ', 19),
 ('природа', 19),
 ('эксперт', 18),
 ('консультация', 18),
 ('ребёнок', 18),
 ('доход', 17),
 ('продукт', 16),
 ('работа', 16),
 ('дом', 16),
 ('эко', 15),
 ('заказ', 15),
 ('партнёр', 14),
 ('любовь', 14),
 ('предприниматель', 14),
 ('вопрос', 13),
 ('тур', 13),
 ('спорт', 13),
 ('опыт', 12),
 ('страна', 12),
 ('команда', 12),
 ('экскурсия', 12),
 ('проект', 12),
 ('гид', 11),
 ('покупка', 11),
 ('город', 11),
 ('магазин', 11),
 ('отдых', 11),
 ('консультант', 11),
 ('москва', 11),
 ('программа', 10),
 ('развитие', 10),
 ('компания', 10),
 ('россия', 10),
 ('молодость', 9),
 ('сибирь', 9),
 ('секрет', 9)]

### most frequent words

In [72]:
flat_corpus = [word for doc in tokenized for word in doc]
counter = Counter(flat_corpus)
counter_sorted = dict(sorted(counter.items(),
                             key=lambda item: item[1], reverse=True))
print(len(counter_sorted))
list(counter_sorted.items())[:N_KEYWORDS]


3475


[('здоровье', 79),
 ('жизнь', 79),
 ('байкал', 51),
 ('wellness', 51),
 ('siberian', 49),
 ('красота', 48),
 ('любить', 46),
 ('бизнес', 43),
 ('moscow', 33),
 ('путешествие', 32),
 ('год', 32),
 ('мир', 31),
 ('писать', 31),
 ('мама', 29),
 ('russia', 25),
 ('жить', 25),
 ('человек', 23),
 ('путешествовать', 22),
 ('ваш', 22),
 ('семья', 22),
 ('зож', 21),
 ('мой', 21),
 ('ссылка', 20),
 ('иркутск', 20),
 ('свой', 20),
 ('директ', 19),
 ('природа', 19),
 ('эксперт', 18),
 ('консультация', 18),
 ('ребёнок', 18),
 ('весь', 17),
 ('доход', 17),
 ('ru', 17),
 ('love', 17),
 ('делиться', 17),
 ('продукт', 16),
 ('whatsapp', 16),
 ('работа', 16),
 ('дом', 16),
 ('эко', 15),
 ('здоровый', 15),
 ('заказ', 15),
 ('счастливый', 15),
 ('партнёр', 14),
 ('сибирский', 14),
 ('работать', 14),
 ('любовь', 14),
 ('предприниматель', 14),
 ('вопрос', 13),
 ('тур', 13)]